In [1]:
! pip install qdrant_client langchain_openai langchain_cohere


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from dotenv import load_dotenv
import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.vectorstores import Qdrant
import qdrant_client
from langchain_core.output_parsers import StrOutputParser # parser
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor_cohere =CohereRerank(model="rerank-multilingual-v3.0",cohere_api_key="weyxiNeVkznwJueXMTuj9Mu7JtwTDf5nMfgPA8XF",top_n=10)
# Charger les variables d'environnement
load_dotenv()
# Récupérer les clés API et chemins nécessaires
HF_TOKEN = os.getenv('API_TOKEN')
openAi8key=os.getenv('openAi8key')
CHROMA_PATH = os.path.abspath(f"../{os.getenv('CHROMA_PATH')}")
COLLECTION_CSV = os.getenv('COLLECTION_CSV')
GROQ_TOKEN = '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'
#llm = ChatGroq(model_name='llama-3.1-70b-versatile', api_key=GROQ_TOKEN, temperature=0)
modelName = "gpt-4o-mini"
llm = ChatOpenAI(model_name=modelName, api_key=openAi8key, temperature=0.3)
FILE_TYPES= ['.png', '.jpeg', '.jpg', '.pdf']
modelName2='gemma2-9b-it'
#llama3-8b-8192
# Initialize memory and conversation chain globally
#memory = ConversationBufferMemory()
memory = ConversationBufferWindowMemory( k=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=openAi8key)
def initialize_vectorstore(embeddings, QDRANT_URL, QDRANT_API_KEY, collection_name):
    qdrantClient = qdrant_client.QdrantClient(
        url=QDRANT_URL,
        prefer_grpc=True,
        api_key=QDRANT_API_KEY)
    return Qdrant(qdrantClient, collection_name, embeddings) #, vector_name='vector_params'


def initialize_retriever(llm, vectorstore,metadata_field_info,document_content_description):
    retriever = SelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True,
        memory=ConversationBufferWindowMemory(k=0),
        search_type="mmr",
        search_kwargs={'k':500,'fetch_k':40}
    )
    return retriever

In [3]:

url="https://a08399e1-9b23-417d-bc6a-88caa066bca4.us-east4-0.gcp.cloud.qdrant.io"
api_key= "lJo8SY8JQy7W0KftZqO3nw11gYCWIaJ0mmjcjQ9nFhzFiVamf3k6XA"
collection_name="OpenAI_SELFQ_collection"

document_content_description = "Informations sur le produit, incluant la référence et la description."
metadata_field_info = [
        {
            'name': "Marque",
            'description': "La Marque du produit.",
            'type': "string",
        },
        {
            'name': "Categorie",
            'description': "La Categorie du produit.",
            'type': "string",
        }
    ]
# Initialiser le vectorstore et le retriever

vectorstore = initialize_vectorstore(embeddings,url,api_key,collection_name)
retriever = initialize_retriever(llm, vectorstore,metadata_field_info,document_content_description)

c:\Users\Dell\Desktop\projet\self_querying\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [4]:
prompt_template_rech= """Tu es un Assistant de Reformulation spécialisé dans les requêtes produit :
   *Base de connaissance *:
    - Voici quelques marques de laptops : Dell, HP, Lenovo, Acer, Asus, Microsoft, MSI, Razer, Samsung, Toshiba, Sony (Vaio), Alienware, Gigabyte, Huawei, LG, Xiaomi, Fujitsu, Chuwi, Clevo, Eurocom
   *Instructions*:
    1. Reformule chaque question pour la rendre plus complète ou plus claire, sans enlever la marque mentionnée.
    2. Objectif : Fournis trois reformulations pour chaque question. Chaque reformulation doit rester fidèle à l’original tout en offrant une légère variation dans la phrasing ou la structure.
        Exemple 1:
        - Question : "Laptop Lenovo i7 16GB RAM 512GB SSD 14"
        - Reformulations : 
          1. "Laptop Lenovo avec processeur i7, 16GB de RAM et stockage de 512GB SSD, écran de 14 pouces"
          2. "Laptop Lenovo i7, 16GB RAM, 512GB SSD, taille de l'écran 14 pouces"
          3. "Laptop Lenovo, i7, 16GB de RAM, SSD de 512GB, écran 14 pouces"

        Exemple 2:
        - Question : "21D60011FR lenovo ThinkPad P16 Gen 1 intel i7 16 GB 1 TB"
        - Reformulations : 
          1. "21D60011FR Lenovo ThinkPad P16 Gen 1 avec Intel i7, 16GB de RAM et disque dur de 1TB"
          2. "21D60011FR ThinkPad P16 Gen 1 Lenovo, processeur Intel i7, 16GB RAM, stockage 1TB"
          3. "21D60011FR Lenovo ThinkPad P16 Gen 1, Intel i7, 16GB de RAM, avec 1TB de stockage"

        Exemple 3:
        - Question : "HP Pavilion x360 Intel Core i5 8GB RAM 256GB SSD"
        - Reformulations : 
          1. "HP Pavilion x360 avec processeur Intel Core i5, 8GB RAM, et disque SSD de 256GB"
          2. "HP Pavilion x360 Intel Core i5, 8GB de RAM et 256GB SSD"
          3. "HP Pavilion x360, Core i5, 8GB RAM, 256GB SSD"

        Exemple 4:
        - Question : "MacBook Pro M1 16GB 512GB SSD"
        - Reformulations : 
          1. "MacBook Pro avec puce M1, 16GB de RAM et 512GB SSD"
          2. "MacBook Pro M1, 16GB RAM, 512GB SSD"
          3. "MacBook Pro M1 avec 16GB de RAM et 512GB de stockage SSD"
    3. Pour chaque question,sans afficher la question, affiche directement les reformulations sans indiquer "Reformulation".
    4. liste chaque reformulation sur une ligne distincte.
    5. N'ajoute pas de sauts de ligne entre les reformulations ni entre les question.
  
    Questions:
    {questions}
"""
prompt_template_metier = """Assistant Spécialisé en Reformulation de Requêtes metier 

    *Base de connaissance* :
    Marques de Laptops : Apple, Dell, HP, Lenovo, Acer, Asus, Microsoft, MSI, Razer, Samsung, Toshiba, Sony (Vaio), Alienware, Gigabyte, Huawei, LG, Xiaomi, Fujitsu, Chuwi, Clevo, Eurocom.

    *Synonymes de catégories* :

    Laptop : Laptop, Ordinateur Portable, PC, PC portable.
    All in One : All-in-One, PC Tout-en-Un.
    Poste de Travail : Poste de Travail, PC, Ordinateur, Desktop, unité centrale.
    Station Mobile : Laptop gaming, laptop haute performance.
    Station de Travail : Station de Travail, Workstation, desktop haute performance, desktop gaming, unité centrale gaming.
    Écran : Écran, Moniteur, Monitor.
    Téléphone : Téléphone, Smartphone.
    Imprimante : Imprimante, Printer.
    Instructions de Reformulation :

    *Remplacement des synonymes* :

    Distingue clairement entre les catégories de produits pour éviter toute confusion. Par exemple, "desktop" se réfère à un poste de travail standard, tandis que "desktop haute performance" désigne une station de travail.
    Remplace les synonymes de catégories par la catégorie principale correspondante lors de la reformulation. Par exemple, remplace "PC portable" par "Laptop". Applique cette règle à toutes les catégories listées ci-dessus.

    *Reformulation basée sur un métier spécifique* :

    Identifie les métiers mentionnés dans la question (par exemple, études, design, programmation, etc.).
    Pour chaque métier, génère une question spécifique adaptée aux besoins de ce métier, en générant une description précise du produit idéal pour ce métier. Utilise des caractéristiques typiques nécessaires pour chaque métier.
    pour chaque metier, Conserve les catégories, marques ou modèles spécifiques mentionnés dans la question, en les intégrant dans la reformulation.
    Précise les caractéristiques techniques essentielles telles que le processeur, la RAM, le stockage, la carte graphique et l'écran, sans fournir d'explication supplémentaire.
    Exemple 1 :
    Question : Quel ordinateur est recommandé pour la programmation avec de bonnes performances ?
    Reformulation : Laptop processeur i7 16GB RAM 512GB SSD écran 15".
    Exemple 2 :
    Question : Je cherche un workstation Dell pour le design avec de bonnes spécifications.
    Reformulation : Station de Travail Dell processeur i9 32GB RAM 1TB SSD carte graphique dédiée écran 4K.
    Exemple 3 :
    Question : Quel ordinateur est recommandé pour la programmation avec de bonnes performances et le design graphique ?"
    Reformulations :
    Laptop  processeur i7, 16GB RAM, 512GB SSD, écran 15 pouces.
    Station de Travail  processeur i9, 32GB RAM, 1TB SSD, carte graphique dédiée, écran 4K.
    
    *Conseils supplémentaires* :
    lors de la reformulation comment par la categorie des produit, emsuite les caracteristique sans ajouter pour quelle tache le produit sera utilise
    Assure-toi que la reformulation soit claire et concise, en se concentrant sur les caractéristiques techniques pertinentes.
    Adapte la reformulation en fonction du contexte spécifique du métier, en mettant en avant les éléments qui répondent aux besoins particuliers de ce métier.
    
    
    *Instructions supplémentaires* :

    Ne réponds pas directement aux questions ; reformule uniquement en suivant les instructions ci-dessus.
    Mets chaque question reformulée sur une seule ligne distincte pour assurer une lisibilité optimale.
    Lorsqu'une question contient plusieurs métiers, mets chaque reformulation pour chaque métier sur une ligne distincte.
    N'ajoute pas de saut de ligne entre les questions reformulées pour maintenir la cohérence.
    Questions : {questions}

"""

from langchain.chains import LLMChain

def reformulate_queries_with_llm(queries: list, llm ,prompt_template) -> list:
    # Join the queries into a single string with newline separation
    joined_queries = "\n".join([f"Question: {query}" for query in queries])
    
    # Prepare the prompt
    prompt = PromptTemplate(template=prompt_template, input_variables=["questions"])
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # Execute the chain to get the reformulations
    reformulated_queries = chain.run(questions=joined_queries)
    print("reformulated_queries \n",reformulated_queries)
    # Split the reformulated queries into a list
    reformulated_list = reformulated_queries.strip().split('\n')
    print("reformulated_list\n",reformulated_list)
    return reformulated_list


In [9]:
queries=['je veux des pc pas puissant pour les etudes et des stations de travail pour la programmation']
reformulated_queries = reformulate_queries_with_llm(queries, llm,prompt_template_metier)

reformulated_queries 
 Laptop processeur i5, 8GB RAM, 256GB SSD, écran 14".  
Station de Travail processeur i7, 16GB RAM, 512GB SSD, carte graphique dédiée, écran 27".
reformulated_list
 ['Laptop processeur i5, 8GB RAM, 256GB SSD, écran 14".  ', 'Station de Travail processeur i7, 16GB RAM, 512GB SSD, carte graphique dédiée, écran 27".']


In [10]:
reformulated_queries = reformulate_queries_with_llm(reformulated_queries, llm,prompt_template_rech)         

reformulated_queries 
 Laptop avec processeur i5, 8GB de RAM, 256GB SSD et écran de 14 pouces.  
Laptop i5, 8GB RAM, 256GB SSD, taille d'écran 14 pouces.  
Laptop, processeur i5, 8GB de RAM, SSD de 256GB, écran de 14".  
Station de travail avec processeur i7, 16GB de RAM, 512GB SSD, carte graphique dédiée et écran de 27 pouces.  
Station de travail i7, 16GB RAM, 512GB SSD, carte graphique dédiée, taille de l'écran 27 pouces.  
Station de travail, processeur i7, 16GB de RAM, SSD de 512GB, écran de 27" avec carte graphique dédiée.  
reformulated_list
 ['Laptop avec processeur i5, 8GB de RAM, 256GB SSD et écran de 14 pouces.  ', "Laptop i5, 8GB RAM, 256GB SSD, taille d'écran 14 pouces.  ", 'Laptop, processeur i5, 8GB de RAM, SSD de 256GB, écran de 14".  ', 'Station de travail avec processeur i7, 16GB de RAM, 512GB SSD, carte graphique dédiée et écran de 27 pouces.  ', "Station de travail i7, 16GB RAM, 512GB SSD, carte graphique dédiée, taille de l'écran 27 pouces.  ", 'Station de travail,

In [11]:
compressor_cohere =CohereRerank(model="rerank-multilingual-v3.0",cohere_api_key="weyxiNeVkznwJueXMTuj9Mu7JtwTDf5nMfgPA8XF",top_n=10)

re_rank_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_cohere, base_retriever=retriever
)
context=[[]]
flattened_docs=[]
if not isinstance(reformulated_queries, list):
        reformulated_queries= [reformulated_queries]
print('questions =',reformulated_queries)
# Check if the list is empty or contains only empty strings
if not reformulated_queries or all(q.strip() == '' for q in reformulated_queries):
    print("Please provide at least one valid question.")
else :
    context = re_rank_retriever.batch(reformulated_queries)

print(context)

questions = ['Laptop avec processeur i5, 8GB de RAM, 256GB SSD et écran de 14 pouces.  ', "Laptop i5, 8GB RAM, 256GB SSD, taille d'écran 14 pouces.  ", 'Laptop, processeur i5, 8GB de RAM, SSD de 256GB, écran de 14".  ', 'Station de travail avec processeur i7, 16GB de RAM, 512GB SSD, carte graphique dédiée et écran de 27 pouces.  ', "Station de travail i7, 16GB RAM, 512GB SSD, carte graphique dédiée, taille de l'écran 27 pouces.  ", 'Station de travail, processeur i7, 16GB de RAM, SSD de 512GB, écran de 27" avec carte graphique dédiée.']
[[Document(metadata={'Categorie': 'Ordinateurs / Serveurs', 'Marque': 'MICROSOFT', 'source': '../data/lenovoHP_clean (1).csv', 'row': 24745, '_id': '832d7945-8909-4fa5-85fc-5aa421be36a4', '_collection_name': 'OpenAI_SELFQ_collection', 'relevance_score': 0.9942251}, page_content='Part Number: QZI-00007\nDescription: Microsoft Surface Laptop 5 - Intel Core i5 - 1235U / jusqu\'à 4.4 GHz - Evo - Win 11 Home - Carte graphique Intel Iris Xe - 8 Go RAM - 256 G

In [12]:
context

[[Document(metadata={'Categorie': 'Ordinateurs / Serveurs', 'Marque': 'MICROSOFT', 'source': '../data/lenovoHP_clean (1).csv', 'row': 24745, '_id': '832d7945-8909-4fa5-85fc-5aa421be36a4', '_collection_name': 'OpenAI_SELFQ_collection', 'relevance_score': 0.9942251}, page_content='Part Number: QZI-00007\nDescription: Microsoft Surface Laptop 5 - Intel Core i5 - 1235U / jusqu\'à 4.4 GHz - Evo - Win 11 Home - Carte graphique Intel Iris Xe - 8 Go RAM - 256 Go SSD - 13.5" écran tactile 2256 x 1504 - IEEE 802.11b, IEEE 802.11a, IEEE 802.11g, IEEE 802.11n, IEEE 802.11ac, Blu'),
  Document(metadata={'Categorie': 'Ordinateurs / Serveurs', 'Marque': 'LENOVO', 'source': '../data/lenovoHP_clean (1).csv', 'row': 20804, '_id': '5e797864-d2b4-4ab8-9a9f-9edaa82ca678', '_collection_name': 'OpenAI_SELFQ_collection', 'relevance_score': 0.91804045}, page_content='Part Number: 20VE011AFR\nDescription: Intel Core i5 1135G7 / 2.4 GHz - Win 11 Pro - Iris Xe Graphics - 16 Go RAM - 512 Go SSD NVMe - 15.6" IPS 19

In [32]:
queries=['laptop, processeur i5, 16GB RAM, stockage 256GB SSD, écran 14". ']
compressor_cohere =CohereRerank(model="rerank-multilingual-v3.0",cohere_api_key="weyxiNeVkznwJueXMTuj9Mu7JtwTDf5nMfgPA8XF",top_n=10)

re_rank_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_cohere, base_retriever=retriever
)
context=[[]]
flattened_docs=[]
if not isinstance(queries, list):
        queries= [queries]
print('questions =',queries)
# Check if the list is empty or contains only empty strings
if not queries or all(q.strip() == '' for q in queries):
    print("Please provide at least one valid question.")
else :
    context = re_rank_retriever.batch(queries)

context

questions = ['laptop, processeur i5, 16GB RAM, stockage 256GB SSD, écran 14". ']


[[Document(metadata={'Categorie': 'Ordinateurs / Serveurs', 'Marque': 'APPLE', 'source': '../data/lenovoHP_clean (1).csv', 'row': 17563, '_id': 'cd3316e3-ae9a-41ce-a26e-e8c737aded7d', '_collection_name': 'OpenAI_SELFQ_collection', 'relevance_score': 0.80255556}, page_content='Part Number: Z15W_4568_FR_CTO\nDescription: CTO/MacBook Air 13 pouces Argent/Puce Apple M2 avec CPU 8 coeurs, GPU 10 coeurs et Neural Engine 16 coeurs + 16 Go de memoire unifiée/mémoire unifiée selectionnée avec le processeur/SSD 256 Go/Clavier Anglais américain/USB C 70 W/FinalCut_Pro_&_Logic_Pro'),
  Document(metadata={'Categorie': 'Ordinateurs / Serveurs', 'Marque': 'APPLE', 'source': '../data/lenovoHP_clean (1).csv', 'row': 17564, '_id': 'b60a91b4-a250-4270-afae-35c0266a73a5', '_collection_name': 'OpenAI_SELFQ_collection', 'relevance_score': 0.78874594}, page_content='Part Number: Z15W_4569_FR_CTO\nDescription: CTO/MacBook Air 13 pouces Argent/Puce Apple M2 avec CPU 8 coeurs, GPU 10 coeurs et Neural Engine 16 

In [27]:
from dotenv import load_dotenv
import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_groq import ChatGroq
from sklearn.metrics.pairwise import cosine_similarity
from langchain.memory import ConversationBufferMemory # Import de la mémoire
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.vectorstores import Qdrant
import qdrant_client
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import Docx2txtLoader
import pymupdf4llm , pymupdf,pytesseract as ocr
from langchain_core.output_parsers import StrOutputParser # parser
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor_cohere =CohereRerank(model="rerank-multilingual-v3.0",cohere_api_key="weyxiNeVkznwJueXMTuj9Mu7JtwTDf5nMfgPA8XF",top_n=10)
# Charger les variables d'environnement
load_dotenv()
# Récupérer les clés API et chemins nécessaires
HF_TOKEN = os.getenv('API_TOKEN')
openAi8key=os.getenv('openAi8key')
CHROMA_PATH = os.path.abspath(f"../{os.getenv('CHROMA_PATH')}")
COLLECTION_CSV = os.getenv('COLLECTION_CSV')
GROQ_TOKEN = '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'
#llm = ChatGroq(model_name='llama-3.1-70b-versatile', api_key=GROQ_TOKEN, temperature=0)
modelName = "gpt-4o-mini"
llm = ChatOpenAI(model_name=modelName, api_key=openAi8key, temperature=0.3)
FILE_TYPES= ['.png', '.jpeg', '.jpg', '.pdf']
modelName2='gemma2-9b-it'
#llama3-8b-8192
# Initialize memory and conversation chain globally
#memory = ConversationBufferMemory()
memory = ConversationBufferWindowMemory( k=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=openAi8key)
def initialize_vectorstore(embeddings, QDRANT_URL, QDRANT_API_KEY, collection_name):
    qdrantClient = qdrant_client.QdrantClient(
        url=QDRANT_URL,
        prefer_grpc=True,
        api_key=QDRANT_API_KEY)
    return Qdrant(qdrantClient, collection_name, embeddings) #, vector_name='vector_params'


def initialize_retriever(llm, vectorstore,metadata_field_info,document_content_description):
    retriever = SelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True,
        memory=ConversationBufferWindowMemory(k=0),
        search_type="similarity_score_threshold",
        search_kwargs={'score_threshold':0.4, 'k':25}
    )
    return retriever
async def query_bot(retriever,question,prompt):
    context = retriever.invoke(question)
    if not context:
        return "Je n'ai pas trouvé de produits correspondants."

    #query_embedding = embedding_function.embed_query(question)
    #doc_embeddings = [embedding_function.embed_query(doc.page_content) for doc in context]
    #similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    document_chain = create_stuff_documents_chain(llm, prompt)
              
    # Charger l'historique des conversations
    conversation_history = memory.load_memory_variables({})
    result = document_chain.invoke(
            {
                "context": context,
                "historique":conversation_history['history'],
                "question": question  # Utiliser 'question' au lieu de 'messages'
            },
    )
    # Save context
    memory.save_context({"question": question}, {"response": result})
    

    return result

#Unique union of retrieved docs
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]
def batch_query_bot(retriever, questions: list[str] | str, prompt):
    re_rank_retriever = ContextualCompressionRetriever(
        base_compressor=compressor_cohere, base_retriever=retriever
    )
    context = [[]]
    flattened_docs = []
    if not isinstance(questions, list):
        questions = [questions]
    print('questions =', questions)
    
    if not questions or all(q.strip() == '' for q in questions):
        yield "Please provide at least one valid question."
        return
    
    context = re_rank_retriever.batch(questions)
    
    if not context:
        yield "Je n'ai pas trouvé de produits correspondants."
        return
    
    liste = get_unique_union(context)
    print(f"Nombre total de liste : ", {len(liste)})

    document_chain = create_stuff_documents_chain(llm, prompt)
    conversation_history = memory.load_memory_variables({})
    
    # Simulate streaming by iterating over results
    for result in document_chain.stream({
        "context": liste,
        "historique": conversation_history['history'],
        "question": questions
    }):
        yield result

    memory.save_context({"question": questions},{"response":result})


In [28]:

from langchain_groq import ChatGroq
import asyncio
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser # parser
import pymupdf, pymupdf4llm
import pytesseract 
from langchain_core.prompts import PromptTemplate
import os
import uuid, base64
from PIL import Image
from io import BytesIO
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.chains import LLMChain
import extract_msg
from email.parser import BytesParser
from email import policy
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

load_dotenv()

# Set your OpenAI API key
openAi8key=os.getenv('openAi8key')
# Charger la fonction d'embedding
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=openAi8key)
file_up_key= uuid.uuid4().hex
# Initialize the chat model
modelName = "gpt-4o-mini"
llm = ChatOpenAI(model_name=modelName, api_key= openAi8key, temperature=0.5)
# Initialiser le modèle LLM
GROQ_TOKEN = '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'

def llm_generation(modelName, GROQ_TOKEN):         
    llm = ChatGroq(model_name=modelName, api_key=GROQ_TOKEN, temperature=0)
    return llm


url="https://a08399e1-9b23-417d-bc6a-88caa066bca4.us-east4-0.gcp.cloud.qdrant.io"
api_key= "lJo8SY8JQy7W0KftZqO3nw11gYCWIaJ0mmjcjQ9nFhzFiVamf3k6XA"
collection_name="OpenAI_SELFQ_collection"
FILE_TYPES= ['png', 'jpeg', 'jpg', 'pdf', 'docx', 'xlsx','msg','eml']
modelName2='gemma2-9b-it'
modelName = "gpt-4o-mini"
#llm2= llm_generation(modelName2,GROQ_TOKEN)
llm2 = ChatOpenAI(model_name=modelName, api_key= openAi8key, temperature=0.3)

pdf_prompt_instruct =  """Tu es un **Assistant Extraction de Descriptions :**
1. **Extraction** : Identifie toutes les descriptions de produits dans le contexte.
2. **Reformulation** : Si nécessaire, reformule les descriptions tout en restant fidèle à l'original.
3. **Contexte Vide** : Ne génère aucune réponse si le contexte est vide ou insuffisant.
4. **Format** : un produit decrit par ligne et une ligne suffit un produit, numérotée.
5. **Réponse Brute** : Retourne uniquement les descriptions sans commentaire.
6. **Nombre**: N'oublie aucun produit.
{contexte}
Reponse:"""
# image prompt
image_system_prompt= """ Etant donné l'image ci-dessous, TU fais :
1. **IDENTIFIES** toutes les descriptions completes des produits dans l'image.
2. **Ne GENERE aucune** réponse si l'image n'est pas claire. Retourne "image pas claire" dans ce cas.
3. **Format** : un produit décrit par ligne et une ligne suffit pour un produit, numérotée.
4. **CONTENU**: doit contenir que la description technique pas d'info supplementaire
4. **SANS COMMENTAIRES**
"""
image_user_prompt=  [
                        {
                            "type": "image_url",
                            "image_url":   {"url": "data:image/{img_format};base64,{image_data}",
                                            "detail": "low"}
                        }
                    ]

image_prompt= ChatPromptTemplate.from_messages(
    messages= [
        ('system', image_system_prompt),
        ('user', image_user_prompt)
    ]
)

pdf_prompt = PromptTemplate.from_template(pdf_prompt_instruct)
pdf_chain=  pdf_prompt | llm2 | StrOutputParser()
image_chain= image_prompt | llm | StrOutputParser()

# Créer un template de prompt pour la reformulation
prompt_template = """Tu es un Assistant de Reformulation spécialisé dans les requêtes produit :
   *Base de connaissance *:
    - voici quelque marque de laptops : Dell, HP, Lenovo, Acer, Asus, Microsoft, MSI, Razer, Samsung, Toshiba, Sony (Vaio), Alienware, Gigabyte, Huawei, LG, Xiaomi, Fujitsu, Chuwi, Clevo, Eurocom
   *Instructions*:
    1. Vérifie si la question inclut une marque. Si oui, reformule la question selon les instructions ci-dessous ; sinon, retourne la question telle qu'elle est.
    2. Objectif : Reformuler chaque requête de manière complète, auto-suffisante, en excluant  la marque mentionnée s'il y en a une. Fournis trois reformulations pour chaque question, en gardant les reformulations proches de l'originale.
        Exemple 1:
        - Question : "Laptop Lenovo i7 16GB RAM 512GB SSD 14" "
        - Reformulations : 
          1. "Laptop i7 16GB RAM 512GB SSD 14" 
          2. "Laptop i7 16GB RAM avec 512GB SSD 14 pouces"
          3. "Laptop i7 16GB RAM 512GB SSD 14 pouces"
        Exemple 2:
        - Question : " 21D60011FR lenovo ThinkPad P16 Gen 1 intel i7 16 GB 1 TB"
        - Reformulations : 
          1. "21D60011FR ThinkPad P16 Gen 1 intel i7 16 GB 1 TB"
          2. "21D60011FR ThinkPad P16 Gen 1 intel i7 16 GB 1 To"
          3. "21D60011FR ThinkPad P16 Gen 1 intel i7 16 GB avec 1 To"
    3. Pas de Réponse : Ne réponds jamais à la question, reformule-la uniquement si nécessaire.
    4. Met chaque question reformulée sur une ligne, en indiquant les reformulations numérotées.
    5. N'ajoute pas des saut de ligne entre les questions reformulées.
    Questions:
    {questions}
"""
prompt_template_rech= """Tu es un Assistant de Reformulation spécialisé dans les requêtes produit :
   *Base de connaissance *:
    - Voici quelques marques de laptops : Dell, HP, Lenovo, Acer, Asus, Microsoft, MSI, Razer, Samsung, Toshiba, Sony (Vaio), Alienware, Gigabyte, Huawei, LG, Xiaomi, Fujitsu, Chuwi, Clevo, Eurocom
   *Instructions*:
    1. Reformule chaque question pour la rendre plus complète ou plus claire, sans enlever la marque mentionnée.
    2. Objectif : Fournis trois reformulations pour chaque question. Chaque reformulation doit rester fidèle à l’original tout en offrant une légère variation dans la phrasing ou la structure.
        Exemple 1:
        - Question : "Laptop Lenovo i7 16GB RAM 512GB SSD 14"
        - Reformulations : 
          1. "Laptop Lenovo avec processeur i7, 16GB de RAM et stockage de 512GB SSD, écran de 14 pouces"
          2. "Laptop Lenovo i7, 16GB RAM, 512GB SSD, taille de l'écran 14 pouces"
          3. "Laptop Lenovo, i7, 16GB de RAM, SSD de 512GB, écran 14 pouces"

        Exemple 2:
        -"21D60011FR lenovo ThinkPad P16 Gen 1 intel i7 16 GB 1 TB"
        - "21D60011FR Lenovo ThinkPad P16 Gen 1 avec Intel i7, 16GB de RAM et disque dur de 1TB","21D60011FR ThinkPad P16 Gen 1 Lenovo, processeur Intel i7, 16GB RAM, stockage 1TB","21D60011FR Lenovo ThinkPad P16 Gen 1, Intel i7, 16GB de RAM, avec 1TB de stockage"

        Exemple 3:
        - "HP Pavilion x360 Intel Core i5 8GB RAM 256GB SSD"
        -"HP Pavilion x360 avec processeur Intel Core i5, 8GB RAM, et disque SSD de 256GB","HP Pavilion x360 Intel Core i5, 8GB de RAM et 256GB SSD","HP Pavilion x360, Core i5, 8GB RAM, 256GB SSD"

        Exemple 4:
        - "MacBook Pro M1 16GB 512GB SSD"
        -"MacBook Pro avec puce M1, 16GB de RAM et 512GB SSD","MacBook Pro M1, 16GB RAM, 512GB SSD","MacBook Pro M1 avec 16GB de RAM et 512GB de stockage SSD"
    Ne réponds pas directement aux questions ; reformule uniquement en suivant les instructions ci-dessus.
    Mets chaque question reformulée sur une seule ligne pour garantir une lisibilité optimale.
    N'ajoute pas de saut de ligne entre les questions reformulées pour maintenir la cohérence.
    Questions:
    {questions}
"""
prompt_template_metier = """Assistant Spécialisé en Reformulation de Requêtes metier 
    *Base de connaissance* :
    Marques de Laptops : Apple, Dell, HP, Lenovo, Acer, Asus, Microsoft, MSI, Razer, Samsung, Toshiba, Sony (Vaio), Alienware, Gigabyte, Huawei, LG, Xiaomi, Fujitsu, Chuwi, Clevo, Eurocom.

    *Synonymes de catégories* :

    Laptop : Laptop, Ordinateur Portable, PC, PC portable.
    All in One : All-in-One, PC Tout-en-Un.
    Poste de Travail : Poste de Travail, PC, Ordinateur, Desktop, unité centrale.
    Station Mobile : Laptop gaming, laptop haute performance.
    Station de Travail : Station de Travail, Workstation, desktop haute performance, desktop gaming, unité centrale gaming.
    Écran : Écran, Moniteur, Monitor.
    Téléphone : Téléphone, Smartphone.
    Imprimante : Imprimante, Printer.
    Instructions de Reformulation :

    *Remplacement des synonymes* :

    Distingue clairement entre les catégories de produits pour éviter toute confusion. Par exemple, "desktop" se réfère à un poste de travail standard, tandis que "desktop haute performance" désigne une station de travail.
    Remplace les synonymes de catégories par la catégorie principale correspondante lors de la reformulation. Par exemple, remplace "PC portable" par "Laptop". Applique cette règle à toutes les catégories listées ci-dessus.

    *Reformulation basée sur un métier spécifique* :

    Si la question concerne plusieurs types de produits, reformule chaque produit séparément en décrivant ses caractéristiques essentielles. Utilise des phrases concises et informatives.
    Conserve les catégories, marques ou modèles spécifiques mentionnés dans la question.
    Exemple : "Pouvez-vous nous recommander des laptops pour nos développeurs de logiciels, des stations mobiles pour nos graphistes, et des moniteurs pour les postes de travail de notre équipe technique ?"
    Reformulation :

    "Laptop avec processeur i7, 16GB de RAM, 1TB SSD et carte graphique dédiée."
    "Station mobile équipée d'un processeur i7, 32GB de RAM, 1TB SSD et carte graphique dédiée avec un écran 4K."
    "Moniteur de 27 pouces, résolution 1440p et temps de réponse rapide."
    *Conseils supplémentaires* :

    Assure-toi que chaque reformulation soit clairement séparée et autonome.
    Présente chaque reformulation sur une ligne distincte pour une clarté maximale.
    *Instructions supplémentaires* :

    Ne réponds pas directement aux questions ; reformule uniquement selon les instructions ci-dessus.
    Mets chaque reformulation sur une ligne séparée.
    Assure-toi que chaque description de produit est claire et distincte.
    Questions : {questions}



"""

def reformulate_queries_with_llm(queries: list, llm ,prompt_template) -> list:
    # Join the queries into a single string with newline separation
    joined_queries = "\n".join([f"Question: {query}" for query in queries])
    
    # Prepare the prompt
    prompt = PromptTemplate(template=prompt_template, input_variables=["questions"])
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # Execute the chain to get the reformulations
    reformulated_queries = chain.run(questions=joined_queries)
    
    # Split the reformulated queries into a list
    reformulated_list = reformulated_queries.strip().split('\n')
    
    return reformulated_list

def extract_text_from_img(img):
    text = pytesseract.image_to_string(img)
    return text

def extract_text_from_pdf(pdf):
        file= pymupdf.open(pdf)
        p= file[0].get_text()
        text= ''
        if bool(p.strip()):
                text= pymupdf4llm.to_markdown(pdf)
        else:
                for page in file:
                        tp= page.get_textpage_ocr()
                        text += f" {tp.extractTEXT()} \n\n" 
        file.close()
        return text


def extract_text_from_docx_xlsx(docx):
        text= pymupdf4llm.to_markdown(docx)
        return text

#import extract_msg
def extract_text_and_attachments_from_msg(path, temp_dir):
    msg = extract_msg.Message(path)
    # Extraire le corps du message
    msg_message = "le contenu de l'email est : \n\n " + msg.body
    
    msg.close()
    return msg_message
def extract_eml_file(file_path,):
    with open(file_path, 'rb') as f:
        msg = BytesParser(policy=policy.default).parse(f)

    # Extract body
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            disposition = part.get("Content-Disposition")

            # Extract the plain text part
            if content_type == "text/plain" and disposition is None:
                extracted_text = part.get_payload(decode=True).decode(part.get_content_charset())
    else:
        # For non-multipart emails
        extracted_text = msg.get_payload(decode=True).decode(msg.get_content_charset())

    return extracted_text
def extract_text_from_file(file):
    _, ext = os.path.splitext(file)
    ext= ext.strip('.')
    assert ext in FILE_TYPES, f'wrong file type. The file must be one of the following {FILE_TYPES}'
        # print(ext)
    if ext in ['docx', 'xlsx']:
            return extract_text_from_docx_xlsx(file)
    if ext == 'pdf':
            return extract_text_from_pdf(file)
    if ext == 'msg':
        temp_dir = os.path.join('temp_attachments', uuid.uuid4().hex)
        os.makedirs(temp_dir, exist_ok=True)
        msg_text = extract_text_and_attachments_from_msg(file, temp_dir)
        # Nettoyer les fichiers temporaires
        for root, dirs, files in os.walk(temp_dir):
            for name in files:
                os.remove(os.path.join(root, name))
        os.rmdir(temp_dir)
        return msg_text
    if ext =='eml':
        return extract_eml_file(file)
    return parse_image(file)

def encode_image(image_path: str, size: tuple[int, int]= (320, 320)):
    _, ext= os.path.splitext(image_path)
    ext= ext.strip('.').lower()
    
    with open(image_path, 'rb') as f:
        img_f= Image.open(BytesIO(f.read())).resize(size, Image.Resampling.BILINEAR)
    
    buffered= BytesIO()
    if ext in ['jpeg', 'jpg']:
        img_format= 'JPEG'
    else:
        img_format= 'PNG'
    img_f.save(buffered, format= img_format)
    return base64.b64encode(buffered.getvalue()).decode(), img_format

def parse_image(image_path: str, chain, size: tuple[320, 320]):
    _, ext= os.path.splitext(image_path)
    ext= ext.strip('.').lower()
    subtypes= ['png', 'jpg', 'jpeg', 'webp']
    assert ext in subtypes, f'wrong file type. the file must be on of following {subtypes}'
    b64_img, img_format= encode_image(image_path, size)
    
    return chain.invoke({'image_data': b64_img, 'img_format': img_format})
              
def parse_file(filepath, parser1=pdf_chain, parser2=image_chain, size=(320, 320)):
    _, ext= os.path.splitext(filepath)
    ext= ext.strip('.').lower()
    assert ext in FILE_TYPES, f'wrong file type. the file must be on of following {FILE_TYPES}'
    
    if ext in ['pdf', 'docx', 'xlsx','msg','eml']:
        text = extract_text_from_file(filepath)
        return parser1.invoke(text)
    else:
        return parse_image(filepath, parser2, size)
document_content_description = "Informations sur le produit, incluant la référence et la description."
metadata_field_info = [
        {
            'name': "Marque",
            'description': "La Marque du produit.",
            'type': "string",
        },
        {
            'name': "Categorie",
            'description': "La Categorie du produit.",
            'type': "string",
        }
    ]
# Initialiser le vectorstore et le retriever

vectorstore = initialize_vectorstore(embeddings,url,api_key,collection_name)
retriever = initialize_retriever(llm, vectorstore,metadata_field_info,document_content_description)
# Construire le template de prompt
prompt_similarite = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                """
               *Base de savoir*:
                1. *Synonymes de catégories* :
                    - *Laptop* : Laptop, Ordinateur Portable, PC
                    - *All in One*: All-in-One, PC Tout-en-Un
                    - *Poste de Travail* : Poste de Travail, PC, Ordinateur, Desktop, unité centrale
                    - *Station Mobile*: Laptop gaming, laptop haute performance
                    - *Station de Travail*: workstation, desktop haute performance, desktop gaming, unité centrale gaming
                    - *Écran* : Écran, Moniteur, Monitor
                    - *Téléphone* : Téléphone, Smartphone
                    - *Imprimante* : Imprimante, Printer

                2. *Priorités de classement* selon les catégories :

                | Catégorie | Priorité 1 | Priorité 2 | Priorité 3 | Priorité 4 | Priorité 5 | Priorité 6 | Priorité 7 | Priorité 8 |
                |---|---|---|---|---|---|---|---|---|
                | Laptops, All-in-One | Part Number | Modèle | Marque | Écran (taille, résolution, tactile) | CPU (modèle, famille, génération) | RAM / Stockage | Autres | N/A |
                | Station Mobile | Part Number | Modèle | Marque | Écran (taille, résolution, tactile) | CPU (modèle, famille, génération) | GPU | RAM / Stockage | Autres |
                | Desktop | Part Number | Modèle | Marque | CPU (modèle, famille, génération) | Format (tour, SFF, mini, etc.) | RAM / Stockage | Autres | N/A |
                | Workstation | Part Number | Modèle | Marque | CPU (modèle, famille, génération) | Format (tour, SFF, mini, etc.) | GPU | RAM / Stockage | Autres |
                | Imprimante | Part Number | Modèle | Marque | Type (couleur, noir et blanc) | Fonctionnalité (numérisation, recto-verso, photocopie, etc.) | Vitesse (PPM) | Papier (capacité, formats A4, A3, type) | Connectivité (Bluetooth, Wi-Fi, USB) |

                ---

                *Instructions*:
                0. dites combien de produit dans le contexte
                1. Utilisez le contexte fourni entre triple crochets pour répondre à la requête en listant tous les produits. 
                2. Identifiez la catégorie appropriée en utilisant la liste des synonymes.
                3. Classez les produits par ordre décroissant de similarité avec la description de référence donnée, en suivant l'ordre des priorités de 1 à 8, comme le montre l'example ci-dessous entre triple parenthèses.
                4. Si plusieurs produits sont à égalité pour une priorité donnée, passez à la suivante pour affiner le classement.
                5. En cas d'égalité après application de toutes les priorités, maintenez leur ordre d'origine ou utilisez des critères supplémentaires si disponibles.
                6. Utilisez le contexte fourni entre triple accolades pour répondre à la requête en listant les produits. Si le contexte est vide ou que les instructions ne s'appliquent pas, répondez "pas d'équivalents".
                7. Format de réponse : tableau avec les colonnes suivantes:
                    - Référence : Part number
                    - Catégorie : Type de produit (ex. ordinateur, téléphone)
                    - Marque : Marque du produit
                    - Description : Description complète du produit
                8. Si le contexte est vide ou que qu'il ne contient pas les produits equivalents a la question : répondez "pas de produits équivalents".
                *Pas de commentaire*


                ---

                *Exemple* :
                (((la requete est trouver des equivalents pour :\
                    8A4H6EA, HP Dragonfly 13.5 G4. Type de produit: Ordinateur portable, Format: Clapet. Famille de processeur: Intel® Core™ i7, Modèle de processeur: i7-1355U. Taille de l'écran: 34,3 cm (13.5"), Type HD: WUXGA+, Résolution de l'écran: 1920 x 1280 pixels, Écran tactile. Mémoire interne: 16 Go, Type de mémoire interne: LPDDR5-SDRAM. Capacité totale de stockage: 512 Go, Supports de stockage: SSD. Modèle d'adaptateur graphique inclus: Intel Iris Xe Graphics. Système d'exploitation installé: Windows 11 Pro. Couleur du produit: Bleu

                Les priorités seront :

                1. Part Number: - Si un part number (référence) est présent, les produits avec le même part number que la référence sont prioritaires. Donc : Si un produit HP Dragonfly 13.5 G4 avec le part number "8A4H6EA" est trouvé, il sera classé en premier.

                2. Modèle: Ensuite classez les produits avec le même modèle ("Dragonfly G4") en priorité. Donc : Tous les HP Dragonfly G4 seront classés ici.

                3. Marque: Ensuite, classez les produits de la même marque (HP), même si le modèle diffère. Donc : Les autres Laptops HP (par exemple, HP Spectre, HP EliteBook) seront classés ici.

                4. Écran (taille, résolution, tactile): Si aucun produit ne correspond à la marque, listez les produits avec un écran similaire (13.5 pouces, WUXGA+, tactile), même si la marque et le modèle diffèrent. Donc: Laptops d'autres marques avec un écran de 13.5 pouces, WUXGA+ tactile, comme un Dell XPS 13.

                5. CPU (famille, modèle, génération): Si aucun produit ne correspond à l'écran, classez les produits avec le même CPU (Intel Core i7, i7-1355U), même si la marque, le modèle, et l'écran diffèrent. Donc : Laptops avec un Intel Core i7, i7-1355U, indépendamment de la marque et du modèle.

                6. RAM et Stockage: Ensuite, passez aux produits ayant une RAM de 16 Go et un SSD de 512 Go, même si tous les autres critères ne correspondent pas. Donc: Laptops avec 16 Go RAM et 512 Go SSD, sans se soucier de la marque, du modèle, ou du CPU.

                7. Autres: Enfin, considérez les autres spécifications, comme la carte graphique intégrée (Intel Iris Xe Graphics), le système d'exploitation (Windows 11 Pro), ou la couleur (bleu). Donc: Laptops avec Intel Iris Xe Graphics, même s'ils n'ont pas de CPU i7, 16 Go RAM, ou 512 Go SSD.
                )))


                Contexte: {{{context}}}
                historique :{historique}
                Question: {question}

                Réponse :Voici une liste de produits équivalents au vôtre :
                """
            ),
        ]
)
prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                """
                Tu es un assistant vendeur. Tu as accès au contexte seulement. Ne génère pas des informations si elles ne sont pas dans le contexte. 
                Répond seulement si tu as la réponse. affiche les produit un par un, pour chaque produit affiche son nom puis juste en dessous tableau qui contient ces colonne Référence,Categorie, Marque, Description.        
                Il faut savoir que laptop, ordinateur, ordinateurs portable , pc et poste de travail ont tous le même sens.
                Il faut savoir que téléphone portable et smartphone ont le même sens.
                Il faut savoir que tout autre caractéristique du produit tel que la RAM stockage font partie de la description du produit et il faut filtrer selon la marque et la catégorie seulement.
                Si le contexte est vide, dis-moi que tu n'as pas trouvé de produits correspondants. Je veux que la réponse soit claire et facile à lire, avec des sauts de ligne pour séparer chaque produit. Ne me donne pas de produits qui ne sont pas dans le contexte.
                lorsque une question de similarite entre des produits est poser, il faut dabord commencer par les produit qui ont des processeur qui se ressemble le plus, puis la memoire ram , puis le stockage, puis les autres caracteristique
                la question peut contenir  plusieurs produits avec differentes descriptions, il faut chercher sur les differents produits demandé .
                si je te pose une question sur les question ou les reponses fournient precedement tu doit me repondre selon l'historique.
                tu ne doit pas oublier l'historique car parfois le user continue a te poser des question sur tes reponses que tas deja fourni aupatavant

                Contexte: {context}
                historique :{historique}
                Question: {question}

                Réponse :
                """
            ),
        ]
    )
    


In [29]:
from IPython.display import display, Markdown, clear_output
queries=['un PC HP i7']
reformulated_queries = reformulate_queries_with_llm(queries, llm,prompt_template_rech)
print(reformulated_queries)
# Get the bot's response
result= batch_query_bot(retriever,reformulated_queries,prompt)

response_str = ""
for chunk in result:
    response_str += chunk
    clear_output(wait=True)  # Effacez l'affichage précédent
    display(Markdown(response_str))  # Affichez le texte mis à jour

Voici les produits correspondant à votre demande :

**Produit 1 :**  
Part Number: 7B0P5EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 7B0P5EA#ABF       | Tout-en-un  | HP     | Core i7 13700 / 2.1 GHz - vPro - RAM 16 Go - SSD 512 Go - NVMe, TLC - GeForce RTX 3500 Ti (4Go) - Win 11 Pro - moniteur |

**Produit 2 :**  
Part Number: 7B0P3EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 7B0P3EA#ABF       | Tour        | HP     | Core i7 13700 / jusqu'à 5.2 GHz - RAM 16 Go - SSD 512 Go - NVMe - UHD Graphics 770 - Win 11 Pro - moniteur : aucun |

**Produit 3 :**  
Part Number: 5V9F7EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 5V9F7EA#ABF       | Tout-en-un  | HP     | Core i7 12700 / jusqu'à 4.9 GHz - vPro - RAM 16 Go - SSD 512 Go - NVMe, TLC - UHD Graphics 770 - Win 11 Pro |

**Produit 4 :**  
Part Number: 86C36EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 86C36EA#ABF       | SFF         | HP     | Core i7 13700 / jusqu'à 5.2 GHz - RAM 32 Go - SSD 512 Go - NVMe - RTX A2000 - Win 11 Pro - moniteur : aucun |

**Produit 5 :**  
Part Number: 8T1N6EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 8T1N6EA#ABF       | Tour        | HP     | Core i7 i7-14700 / jusqu'à 5.4 GHz - RAM 32 Go - SSD 512 Go - NVMe, TLC - GF RTX 3060 - Win 11 Pro - moniteur : aucun |

**Produit 6 :**  
Part Number: 8T1N5EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 8T1N5EA#ABF       | Tour        | HP     | Core i7 i7-14700 / jusqu'à 5.4 GHz - RAM 16 Go - SSD 512 Go - NVMe, TLC - UHD Graphics 770 - Win 11 Pro - moniteur : aucun |

**Produit 7 :**  
Part Number: A0ZA6EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| A0ZA6EA#ABF       | Tout-en-un  | HP     | Core i7 i7-14700 / jusqu'à 5.4 GHz - vPro - RAM 16 Go - SSD 512 Go - NVMe, TLC - UHD Graphics 770 - Win 11 Pro |

**Produit 8 :**  
Part Number: 8A3F5EA#ABF  
| Référence          | Catégorie   | Marque | Description                                                                                       |
|--------------------|-------------|--------|---------------------------------------------------------------------------------------------------|
| 8A3F5EA#ABF       | Notebook    | HP     | Core i7 - 1355U / jusqu'à 5 GHz - RAM 16 Go - 512 Go SSD NVMe - Win 11 Pro - 14" IPS HP SureView Reflect 1920 x 1200 |

Si vous avez d'autres questions ou besoin de plus d'informations, n'hésitez pas à demander.

In [ ]:
import streamlit as st
st.title("🧠 Sales Smart Assistant DGF")

if "query" not in st.session_state:
    st.session_state.query = " "

if 'messages' not in st.session_state:
    st.session_state.messages = []

if "extracted_text" not in st.session_state:
    print('im in ext text')
    st.session_state.extracted_text =" "
if "file_up_key" not in st.session_state:

    print('im in filu up key')
    st.session_state.file_up_key= uuid.uuid4().hex

with st.sidebar:
    
    st.session_state.query = st.text_area(label='recherche manuelle', placeholder='Comment puis-je vous aider?', height=150)
    uploaded_file =  st.file_uploader("Téléchargez un fichier contenant les produits que vous chercher", type=FILE_TYPES, key=st.session_state.file_up_key)
    if uploaded_file:
        filepath= uploaded_file.name
        data= uploaded_file.read()
        uploaded_file.close()
        uploaded_file= None
        with open(filepath, 'wb') as up_file:
            up_file.write(data)
        st.session_state.extracted_text = parse_file(filepath)
        #os.remove(filepath)
        st.session_state.file_up_key= uuid.uuid4().hex
        st.markdown(st.session_state.extracted_text)
    
    # Ajouter un conteneur pour le bouton
    with st.container():
        if st.button('Rechercher '):
            full_query= f"{st.session_state.query}{st.session_state.extracted_text}"
            if not full_query.strip() :
                st.warning("La requête est vide. Veuillez entrer une requête valide.")
            else :
                # Append the user's input to the chat history
                st.session_state.messages.append({"role": "user", "content": full_query})
                queries = full_query.strip('\n').split('\n')
                # Delete the temporary file
                reformulated_queries = reformulate_queries_with_llm(queries, llm,prompt_template_rech)
                print(reformulated_queries)
                start_time =time.time()
                # Get the bot's response
                result= asyncio.run(batch_query_bot(retriever,reformulated_queries,prompt))
                #print(f"Résultat: {result}, Temps d'exécution: {exec_time} secondes")
                exec_time=time.time() - start_time
                # Append the bot's response to the chat history
                st.session_state.messages.append({"role": "ai","content" :f"{result}\n\n(Temps d'exécution: {exec_time:.2f} secondes)"})
                st.session_state.extracted_text= ""
                st.session_state.query = "" 


#Affichage 
# Create pairs of (question, response) and reverse the list
message_pairs = []
current_pair = []

for message in st.session_state.messages:
    if message["role"] == "user":
        if current_pair:
            message_pairs.append(current_pair)
        current_pair = [message]
    else:
        if current_pair:
            current_pair.append(message)
            message_pairs.append(current_pair)
            current_pair = []

# Handle case if last pair is incomplete
if current_pair:
    message_pairs.append(current_pair)

# Display the conversation in reverse chronological order
for pair in reversed(message_pairs):
    for message in pair:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])